<a href="https://colab.research.google.com/github/Makito042/Domain-Specific-Assistant-via-LLMs-Fine-Tuning/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installation

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install evaluate rouge_score sacrebleu bert_score

## 2. Load and Preprocess Data (Improved)

In [ ]:
import pandas as pd
from datasets import Dataset

def create_improved_dataset(input_file):
    try:
        df = pd.read_csv('/content/sample_data/companion_plants.csv')
    except FileNotFoundError:
        print(f"Error: {input_file} not found. Please upload it to Colab.")
        return None

    # Standardize column names
    df.columns = [c.strip() for c in df.columns]

    plant_data = {}

    for _, row in df.iterrows():
        source = str(row['Source Node']).strip().lower()
        relation = str(row['Link']).strip().lower()
        target = str(row['Destination Node']).strip().lower()

        if source not in plant_data:
            plant_data[source] = {'helps': set(), 'helped_by': set()}
        if target not in plant_data:
            plant_data[target] = {'helps': set(), 'helped_by': set()}

        # Bi-directional logic: If A helps B, then B is helped by A
        if relation == 'helps':
            plant_data[source]['helps'].add(target)
            plant_data[target]['helped_by'].add(source)
        elif relation == 'helped_by':
            plant_data[source]['helped_by'].add(target)
            plant_data[target]['helps'].add(source)

    dataset_data = []
    # STRICTER SYSTEM PROMPT to reduce hallucination
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

    for plant, info in plant_data.items():
        plant_formatted = plant.title()

        # 1. Ask about what it helps
        if info['helps']:
            helps_list = sorted(list(info['helps']))
            helps_str = ", ".join(helps_list)
            instruction = f"What plants does {plant_formatted} help grow better?"
            # concise response style
            response = f"{plant_formatted} is a beneficial companion for: {helps_str}."
            dataset_data.append({
                "instruction": instruction,
                "input": "",
                "output": response,
                "text": alpaca_prompt.format(instruction, "", response)
            })

        # 2. Ask about what helps it
        if info['helped_by']:
            helped_by_list = sorted(list(info['helped_by']))
            helped_by_str = ", ".join(helped_by_list)
            instruction = f"What are the best companion plants for {plant_formatted}?"
            response = f"The best companion plants for {plant_formatted} include: {helped_by_str}."
            dataset_data.append({
                "instruction": instruction,
                "input": "",
                "output": response,
                "text": alpaca_prompt.format(instruction, "", response)
            })

    return Dataset.from_list(dataset_data)

dataset = create_improved_dataset("companion_plants.csv")
if dataset:
    print(f"Improved dataset created with {len(dataset)} examples (includes bi-directional usage).")
    print(dataset[0])

Improved dataset created with 284 examples (includes bi-directional usage).
{'instruction': 'What plants does Alliums help grow better?', 'input': '', 'output': 'Alliums is a beneficial companion for: brassicas, capsicum, carrots, fruit trees, nightshades, peppers, potatoes, tomatoes.', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat plants does Alliums help grow better?\n\n### Input:\n\n\n### Response:\nAlliums is a beneficial companion for: brassicas, capsicum, carrots, fruit trees, nightshades, peppers, potatoes, tomatoes.'}


## 3. Load Model

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Gemma patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Unsloth 2026.2.1 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


## 4. Train (Improved Settings)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3, # INCREASED from 1 epoch (60 steps) to 3 epochs to improve recall
        learning_rate = 1e-4, # LOWERED from 2e-4 to prevent overfitting
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", # Better convergence
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

Map (num_proc=2):   0%|          | 0/284 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 284 | Num Epochs = 3 | Total steps = 108
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,611,648 of 2,525,784,064 (0.78% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,5.367600
2,5.495800
3,5.021700
4,5.192900
5,4.807100
6,3.825400
7,3.950700
8,3.449400
9,3.071800
10,2.888800


TrainOutput(global_step=108, training_loss=1.037104746809712, metrics={'train_runtime': 435.8625, 'train_samples_per_second': 1.955, 'train_steps_per_second': 0.248, 'total_flos': 820885589704704.0, 'train_loss': 1.037104746809712, 'epoch': 3.0})

## 5. Quantitative Evaluation

In [ ]:
import evaluate
import torch
from tqdm import tqdm
import pandas as pd

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def calculate_perplexity(model, tokenizer, dataset):
    model.eval()
    nlls = []
    subset = dataset.select(range(min(len(dataset), 50)))
    for example in tqdm(subset, desc="Calculating Perplexity"):
        encodings = tokenizer(example["text"], return_tensors="pt")
        input_ids = encodings.input_ids.to(model.device)
        target_ids = input_ids.clone()
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            nlls.append(outputs.loss)
    return torch.exp(torch.stack(nlls).mean()).item()

def evaluate_model(model, tokenizer, dataset, num_samples=20):
    FastLanguageModel.for_inference(model)
    references = []
    predictions = []
    prompts = []
    samples = dataset.select(range(min(len(dataset), num_samples)))

    for example in tqdm(samples, desc="Generating Predictions"):
        text = example['text']
        parts = text.split("### Response:\n")
        if len(parts) > 1:
            prompt = parts[0] + "### Response:\n"
            ground_truth = parts[1]
        else:
            continue

        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        if "### Response:\n" in decoded:
            pred = decoded.split("### Response:\n")[-1].strip()
        else:
            pred = decoded.strip()

        predictions.append(pred)
        references.append([ground_truth.strip()])
        prompts.append(prompt)

    if predictions:
        bleu_score = bleu.compute(predictions=predictions, references=references)
        rouge_refs = [r[0] for r in references]
        rouge_score = rouge.compute(predictions=predictions, references=rouge_refs)
        try:
            ppl = calculate_perplexity(model, tokenizer, dataset)
        except: ppl = float('nan')

        print(f"\n=== Evaluation Results ===")
        print(f"BLEU: {bleu_score['bleu']:.4f}")
        print(f"ROUGE-L: {rouge_score['rougeL']:.4f}")
        print(f"Perplexity: {ppl:.4f}")

        pd.set_option('display.max_colwidth', None)
        df_results = pd.DataFrame({
            'Input': [p.split('### Instruction:\n')[1].split('\n\n### Input:')[0][:50] for p in prompts[:5]],
            'Ground Truth': [r[0] for r in references[:5]],
            'Prediction': predictions[:5]
        })
        display(df_results)

evaluate_model(model, tokenizer, dataset)

Calculating Perplexity: 100%|██████████| 50/50 [00:08<00:00,  5.84it/s]


=== Evaluation Results ===
BLEU: 0.2378
ROUGE-L: 0.4728
Perplexity: 1.7741


,Input,Ground Truth,Prediction
0,What plants does Alliums help grow better?,"Alliums is a beneficial companion for: brassicas, capsicum, carrots, fruit trees, nightshades, peppers, potatoes, tomatoes.","Alliums is a beneficial companion for: beans, brassicas, cabbage, cauliflower, corn, cucumbers, eggplant, lettuce, onions, peas, potatoes, radishes, spinach, tomatoes, turnips."
1,What are the best companion plants for Alliums?,"The best companion plants for Alliums include: carrots, marigold, marigolds, mints, pansy, peppermint, spearmint, tarragon, tomatoes.","The best companion plants for Alliums include: chamomile, dill, lavender, marigold, nasturtiums, oregano, sage, tarragon."
2,What are the best companion plants for Fruit Trees,"The best companion plants for Fruit Trees include: alliums, borage, delions, lemon balm, marigolds, marjoram, mustard, nasturtium, nasturtiums, parsnip, southernwood, tansy.","The best companion plants for Fruit Trees include: alliums, asparagus, beans, brassicas, broccoli, cabbage, cauliflower, corn, cucurbits, dill, garlic, horseradish, legumes, lettuce, marigolds, onions, peas, radishes, spinach, strawberries, tarragon, tomatoes, turnips, wheat."
3,What are the best companion plants for Nightshades,"The best companion plants for Nightshades include: alliums, basil, carrots, mints, oregano, tarragon.","The best companion plants for Nightshades include: alliums, asparagus, beets, brassicas, chamomile, corn, cucumbers, dill, fennel, garlic, horseradish, lavender, lettuce, marigolds, nasturtiums, onions, peas, radishes, spinach, tarragon, tarragon, tomatoes, turnips, wheat."
4,What plants does Tomatoes help grow better?,"Tomatoes is a beneficial companion for: alliums, asparagus, brassicas, broccoli, cabbage, celery, onion, peppers, roses.","Tomatoes is a beneficial companion for: beans, brassicas, cucumbers, cucurbits, lettuce, legumes, peppers, radishes, spinach, tomatoes."


## 6. Save Model

In [ ]:
model.save_pretrained("lora_model_improved")
!zip -r lora_model_improved.zip lora_model_improved

  adding: lora_model_improved/ (stored 0%)
  adding: lora_model_improved/adapter_model.safetensors (deflated 8%)
  adding: lora_model_improved/adapter_config.json (deflated 58%)
  adding: lora_model_improved/README.md (deflated 65%)
